# Time

In [10]:
import pandas as pd

## Database connection

In [11]:
import yaml
from sqlalchemy import create_engine


with open("../config.yml", "r") as file:
	config = yaml.safe_load(file)
	config_OLTP = config["OLTP"]
	config_OLAP = config["OLAP"]


url_OLTP = (f"{config_OLTP['drivername']}://{config_OLTP['user']}:{config_OLTP['password']}"
			f"@{config_OLTP['host']}:{config_OLTP['port']}/{config_OLTP['database_name']}")

url_OLAP = (f"{config_OLAP['drivername']}://{config_OLAP['user']}:{config_OLAP['password']}"
			f"@{config_OLAP['host']}:{config_OLAP['port']}/{config_OLAP['database_name']}")

OLTP_connection = create_engine(url_OLTP)
OLAP_connection = create_engine(url_OLAP)

## Extraction and transformation

In [12]:
time_dimension = pd.DataFrame({
	"date": pd.date_range(start="18/9/2023", end="1/9/2024", freq="h")
})

time_dimension.head()
print(time_dimension.shape)

(2713, 1)


In [13]:
time_dimension["year"] = time_dimension["date"].dt.year
time_dimension["month"] = time_dimension["date"].dt.month
time_dimension["day"] = time_dimension["date"].dt.day
time_dimension["day_of_year"] = time_dimension["date"].dt.day_of_year
time_dimension["day_of_month"] = time_dimension["date"].dt.days_in_month
time_dimension["month_str"] = time_dimension["date"].dt.month_name()
time_dimension["day_str"] = time_dimension["date"].dt.day_name()

time_dimension.head()

,date,year,month,day,day_of_year,day_of_month,month_str,day_str
0,2023-09-18 00:00:00,2023,9,18,261,30,September,Monday
1,2023-09-18 01:00:00,2023,9,18,261,30,September,Monday
2,2023-09-18 02:00:00,2023,9,18,261,30,September,Monday
3,2023-09-18 03:00:00,2023,9,18,261,30,September,Monday
4,2023-09-18 04:00:00,2023,9,18,261,30,September,Monday


## Load

In [14]:
time_dimension.to_sql("TIME_DIMENSION", OLAP_connection, if_exists="replace", index_label="time_id")

713